In [17]:
import tarfile
import shutil
import os
import cv2
import pathlib
import numpy as np
import pandas as pd

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.resnet import ResNet101

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix

In [18]:
random_seed = 42
keras.utils.set_random_seed(random_seed)

In [42]:
train_data_dir = pathlib.Path('/kaggle/input/foodx-251/clean_train80_sampling_diviso/clean_train80_sampling_diviso')
shuffle_value = True
validation_split = 0.2
batch_size = 256
seed_train_validation = 42

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    labels="inferred",
    label_mode="categorical",
    validation_split=validation_split,
    seed=seed_train_validation,
    subset="training",
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    labels="inferred",
    label_mode="categorical",
    validation_split=validation_split,
    seed=seed_train_validation,
    subset="validation",
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

test_data_dir = pathlib.Path('/kaggle/input/foodx-251/val_set_diviso/val_set_diviso')
shuffle_value = True

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_dir,
    labels="inferred",
    label_mode="categorical",
    subset=None,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

test_degraded_data_dir = pathlib.Path('/kaggle/input/foodx-251/val_set_degraded_diviso/val_set_degraded_diviso')
shuffle_value = True
test_degraded_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_degraded_data_dir,
    labels="inferred",
    label_mode="categorical",
    subset=None,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

Found 75300 files belonging to 251 classes.
Using 60240 files for training.
Found 75300 files belonging to 251 classes.
Using 15060 files for validation.
Found 11993 files belonging to 251 classes.
Found 11993 files belonging to 251 classes.


In [43]:
base_eff_model = EfficientNetB0(weights="imagenet", include_top=True, input_shape=(224,224,3))
base_eff_model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_4 (Rescaling)     (None, 224, 224, 3)          0         ['input_3[0][0]']             
                                                                                                  
 normalization_2 (Normaliza  (None, 224, 224, 3)          7         ['rescaling_4[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_5 (Rescaling)     (None, 224, 224, 3)          0         ['normalization_2

In [44]:
trainable_layer = False
for layer in base_eff_model.layers:
    if layer.name == "block7a_expand_conv":
        trainable_layer=True
    layer.trainable=trainable_layer

eff_model = Model(inputs=base_eff_model.input, outputs=base_eff_model.get_layer('top_dropout').output)
last_layer = base_eff_model.get_layer('top_dropout').output
flatten_layer = Flatten()(last_layer)
classification_layer = Dense(251, activation='softmax')(flatten_layer)
eff_model = Model(inputs=eff_model.input, outputs=classification_layer)

In [45]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
eff_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3)])
history = eff_model.fit(train_ds, epochs=20, verbose=1, validation_data=val_ds)

Epoch 1/20


2024-01-06 19:12:15.993098: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_15/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


236/236 [==============================] - 200s 793ms/step - loss: 5.1923 - accuracy: 0.0633 - top_k_categorical_accuracy: 0.1226 - val_loss: 4.4805 - val_accuracy: 0.1785 - val_top_k_categorical_accuracy: 0.3120
Epoch 2/20
236/236 [==============================] - 156s 657ms/step - loss: 4.1266 - accuracy: 0.2320 - top_k_categorical_accuracy: 0.3774 - val_loss: 3.7562 - val_accuracy: 0.2956 - val_top_k_categorical_accuracy: 0.4465
Epoch 3/20
236/236 [==============================] - 156s 656ms/step - loss: 3.5993 - accuracy: 0.3153 - top_k_categorical_accuracy: 0.4734 - val_loss: 3.4997 - val_accuracy: 0.3408 - val_top_k_categorical_accuracy: 0.4946
Epoch 4/20
236/236 [==============================] - 156s 656ms/step - loss: 3.3573 - accuracy: 0.3570 - top_k_categorical_accuracy: 0.5164 - val_loss: 3.3629 - val_accuracy: 0.3692 - val_top_k_categorical_accuracy: 0.5204
Epoch 5/20
236/236 [==============================] - 156s 656ms/step - loss: 3.2075 - accuracy: 0.3834 - top_k_cat

In [46]:
eff_model.evaluate(test_ds, batch_size=256)
eff_model.evaluate(test_degraded_ds, batch_size=256)

47/47 [==============================] - 24s 468ms/step - loss: 3.2163 - accuracy: 0.3757 - top_k_categorical_accuracy: 0.5277


[3.216282844543457, 0.3757191598415375, 0.5277245044708252]

In [47]:
eff_model.save('efficent_20ep_256batch_sgd_drop.keras')